In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
#from datetime import datetime
import calendar
import datetime
import pickle

In [2]:
df = pd.read_csv("invoices_tester.csv")
df.drop(columns=['itemName', 'itemDesc'], inplace=True)
df

invoiceID  customerID  itemID  itemPrice  itemQTY  totalCost  \
0             1           1  111121      34.99       35    1224.65   
1             2           2  111148       9.99       83     829.17   
2             3           3  111155       9.99       30     299.70   
3             4           4  111145       7.99       40     319.60   
4             5           5  111122      14.99       94    1409.06   
...         ...         ...     ...        ...      ...        ...   
4470       4471        4471  111135      12.99       98    1273.02   
4471       4472        4472  111121      34.99       75    2624.25   
4472       4473        4473  111137       3.99       68     271.32   
4473       4474        4474  111132      59.99       47    2819.53   
4474       4475        4475  111159     129.99       80   10399.20   

     dateInvoiced  
0      2022-01-01  
1      2022-01-01  
2      2022-01-02  
3      2022-01-02  
4      2022-01-02  
...           ...  
4470   2024-03-29  
4471   2024-03-29  
4472   2024-03-29  
4473   2024-03-29  
4474   2024-03-29  

[4475 rows x 7 columns]

In [3]:
df.index = pd.to_datetime(df['dateInvoiced'])
df.drop(columns=['dateInvoiced'], inplace=True)
df.index = pd.DatetimeIndex(df.index).to_period('D')
df

invoiceID  customerID  itemID  itemPrice  itemQTY  totalCost
dateInvoiced                                                              
2022-01-01            1           1  111121      34.99       35    1224.65
2022-01-01            2           2  111148       9.99       83     829.17
2022-01-02            3           3  111155       9.99       30     299.70
2022-01-02            4           4  111145       7.99       40     319.60
2022-01-02            5           5  111122      14.99       94    1409.06
...                 ...         ...     ...        ...      ...        ...
2024-03-29         4471        4471  111135      12.99       98    1273.02
2024-03-29         4472        4472  111121      34.99       75    2624.25
2024-03-29         4473        4473  111137       3.99       68     271.32
2024-03-29         4474        4474  111132      59.99       47    2819.53
2024-03-29         4475        4475  111159     129.99       80   10399.20

[4475 rows x 6 columns]

In [4]:
items = df['itemID'].unique()
print(items)

[111121 111148 111155 111145 111122 111146 111113 111135 111160 111137
 111141 111115 111142 111111 111157 111152 111134 111158 111126 111131
 111144 111130 111129 111150 111114 111151 111119 111156 111138 111127
 111123 111140 111120 111132 111143 111112 111153 111154 111125 111147
 111116 111159 111139 111136 111133 111117 111124 111149 111118 111128]


In [5]:
dfList=[]

for ID in items:
    temp = df[df['itemID'] == ID].copy()
    dfList.append(temp)

In [6]:
splitY=[]

for item in dfList:
    splitY.append(item[['itemQTY']])

In [7]:
dateRange = []
extendedSplit = []
for split in splitY:
    dateRange.append(pd.date_range(start=split.index[0].to_timestamp(), end=split.index[-1].to_timestamp()))
    
for ranges in dateRange:
    extended_split_data = pd.DataFrame(index=pd.concat([pd.Series(ranges)]))
    extendedSplit.append(extended_split_data)
    
for i, split in enumerate(extendedSplit):
    for index in split.index:
        # Check if the index exists in the itemQTY dataframe
        if index in splitY[i].index:
            split.loc[index, 'itemQTY'] = splitY[i].loc[index].values[0]
        else:
            split.loc[index, 'itemQTY'] = 0

   

print(extendedSplit[0])

            itemQTY
2022-01-01     35.0
2022-01-02      0.0
2022-01-03      0.0
2022-01-04      0.0
2022-01-05      0.0
...             ...
2024-03-25      0.0
2024-03-26     66.0
2024-03-27      0.0
2024-03-28      0.0
2024-03-29     75.0

[819 rows x 1 columns]


In [8]:
#enable for testing

#df_dropped_last_30 = []

#for split in extendedSplit:
    #print(split)
    #df_dropped_last_30.append(split.iloc[-30:])
    #split.drop(split.tail(30).index, inplace=True)
    #print(split)

In [9]:
models = []
fitted = []
i = 0

for split in extendedSplit:
    split.index = pd.DatetimeIndex(split.index).to_period('D')
    model = SARIMAX(split, order=(10, 0, 0), trend='ct')
    models.append(model)
    fitted.append(model.fit(maxiter=200))
    print(i)
    i += 1
print(fitted[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [10]:
with open('fitted.pkl', 'wb') as f:
        pickle.dump(fitted, f)
    
with open('items.pkl', 'wb') as f:
    pickle.dump(items, f)

In [11]:
forecast = []
for fit in fitted:
    forecast.append(fit.forecast(steps=30))


#print(forecast)

In [12]:
holderFore = []
i=0
for fore in forecast:
    holderPred = 0
    for pred in fore:
        holderPred += pred
    holderFore.append(round(holderPred))
    i += 1
    
print(holderFore)

[144, 256, 202, 222, 178, 142, 223, 232, 150, 112, 182, 168, 216, 234, 93, 195, 241, 142, 124, 180, 150, 295, 243, 209, 132, 181, 160, 201, 133, 205, 250, 212, 191, 244, 180, 202, 180, 140, 197, 104, 213, 223, 214, 193, 185, 201, 106, 265, 231, 229]


In [13]:
#enable for testing

#holderReal = []
#i=0
#for real in df_dropped_last_30:
   # holderReal.append(0)
   #for qty in real['itemQTY']:
   #     holderReal[i] += qty
   # i += 1
    
#print(holderReal)


In [14]:
#enable for testing

#l=0
#while l < len(items):
    #mae = mean_absolute_error(df_dropped_last_30[l], forecast[l])
    #print(f"Mean Absolute Error: {mae}")
    #l += 1

In [15]:
collected = []
for item, holderPred in zip(items, holderFore):
    collected.append([item, holderPred])
print(collected)

[[111121, 144], [111148, 256], [111155, 202], [111145, 222], [111122, 178], [111146, 142], [111113, 223], [111135, 232], [111160, 150], [111137, 112], [111141, 182], [111115, 168], [111142, 216], [111111, 234], [111157, 93], [111152, 195], [111134, 241], [111158, 142], [111126, 124], [111131, 180], [111144, 150], [111130, 295], [111129, 243], [111150, 209], [111114, 132], [111151, 181], [111119, 160], [111156, 201], [111138, 133], [111127, 205], [111123, 250], [111140, 212], [111120, 191], [111132, 244], [111143, 180], [111112, 202], [111153, 180], [111154, 140], [111125, 197], [111147, 104], [111116, 213], [111159, 223], [111139, 214], [111136, 193], [111133, 185], [111117, 201], [111124, 106], [111149, 265], [111118, 231], [111128, 229]]
